# Churn Prediction


Install packages


In [ ]:
!uv pip install -q \
    requests==2.32.5

Append notebooks directory to sys.path


In [ ]:
import sys

sys.path.append("../../../..")

Import packages


In [ ]:
import requests

load_dotenv()  # Root directory .env file

True

Training pipeline

```py title='app/train.py'
--8<-- "docs/notebooks/python/mlops/deployment-fastapi-churn-prediction/app/train.py"
```


Run pipeline


In [58]:
!cd app && \
    uv run train.py

Application code

```py title='app/main.py'
--8<-- "docs/notebooks/python/mlops/deployment-fastapi-churn-prediction/app/main.py"
```

Dockerfile

```dockerfile title='app/Dockerfile'
--8<-- "docs/notebooks/python/mlops/deployment-fastapi-churn-prediction/app/Dockerfile"
```

Dependencies

```toml title='app/pyproject.toml'
--8<-- "docs/notebooks/python/mlops/deployment-fastapi-churn-prediction/app/pyproject.toml"
```


Build docker image


In [59]:
!cd app && \
    docker build -t fastapi-predict-churn --no-cache .


[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.2s (1/3)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.1s
 => => transferring dockerfile: 368B                                       0.0s
 => [internal] load metadata for ghcr.io/astral-sh/uv:latest               0.2s
 => [internal] load metadata for docker.io/library/python:3.11-slim-bookw  0.2s
[+] Building 0.4s (1/3)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.1s
 => => transferring dockerfile: 368B                                       0.0s
 => [internal] load metadata for ghcr.io/astral-sh/uv:latest               0.3s
 => [internal] load metadata for docker.io/library/python:3.11-slim-bookw  0.3s
[+] Building 0.5s (1/3)                                          docker:default
 => [internal] load build definition fr

Run application


In [60]:
!docker run --rm -it -d -p 9696:9696 fastapi-predict-churn

683b9f9de930162f7287c570e3cbb3b14615d41ef2ef68d9a0d07287020f993a


Check running applications


In [69]:
!docker ps

CONTAINER ID   IMAGE                   COMMAND                  CREATED          STATUS          PORTS                                         NAMES
c916c50b62cd   fastapi-predict-churn   "uvicorn main:app --…"   38 seconds ago   Up 38 seconds   0.0.0.0:9696->9696/tcp, [::]:9696->9696/tcp   modest_liskov


Request application


In [ ]:
url = "http://localhost:9696/predict"

customer = {
    "gender": "female",
    "seniorcitizen": 0,
    "partner": "yes",
    "dependents": "no",
    "phoneservice": "no",
    "multiplelines": "no_phone_service",
    "internetservice": "dsl",
    "onlinesecurity": "no",
    "onlinebackup": "yes",
    "deviceprotection": "no",
    "techsupport": "no",
    "streamingtv": "no",
    "streamingmovies": "no",
    "contract": "month-to-month",
    "paperlessbilling": "yes",
    "paymentmethod": "electronic_check",
    "tenure": 1,
    "monthlycharges": 29.85,
    "totalcharges": 29.85,
}

response = requests.post(url, json=customer)

predictions = response.json()

print(predictions)

{'churn_probability': 0.6638167617162171, 'churn': True}


Stop container


In [70]:
!docker stop c916c50b62cd

c916c50b62cd


Delete container image


In [71]:
!docker rmi fastapi-predict-churn

Untagged: fastapi-predict-churn:latest
Deleted: sha256:ace52a950aabd9dad138cecebf4413525f1706e5a43d7e38beba066db1e334be
